# Training for the utilitarianism task on the reformulated ethics dataset

Paper in which the dataset was released: https://arxiv.org/abs/2008.02275 (ICLR, Hendrycks et al., 2021)

We train on various splits of the reformulated datasets to investigate their effects on performance, and assess the calibration of the reformulated dataset.

## Setup

For Google Colab only:

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

# change to directory containing relevant files
%cd '/content/drive/MyDrive/Colab Notebooks/NLP_CW2/github'

In [ ]:
#@title Run to check we're in the correct directory

try:
    f = open("check_directory.txt")
    print('Success :)')
except IOError:
    print("Wrong directory, please try again")

In [ ]:
from pathlib import Path

OUTPUT_DIR = '6_new_models/outputs'

SAVE_FIGS = True # If true, Figures will be saved (overwriting previous versions) as cells are run
FIG_DIR = f"{OUTPUT_DIR}/figure_outputs"

RUN_DIR = f"{OUTPUT_DIR}/run_outputs"

MODEL_DIR = f"{OUTPUT_DIR}/model_outputs"

for dir in [OUTPUT_DIR, FIG_DIR, RUN_DIR, MODEL_DIR]:
  Path(dir).mkdir(parents=True, exist_ok=True)

In [ ]:
%%capture

# Installs

!git clone https://github.com/hendrycks/ethics.git 

!pip install torch transformers pytorch-transformers # ML
!pip install numpy matplotlib pandas scikit-learn # Data

# Imports

from ethics.utils import get_ids_mask, get_tokenizer, load_model
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
from sklearn.model_selection import ParameterGrid
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
# Data loading functions

def load_util_sentences(data_dir, name="train"):
    path = os.path.join(data_dir, "{}.csv".format(name))
    df = pd.read_csv(path, header=None)
    sentences = []
    for i in range(df.shape[0]):
        sentences.append(df.iloc[i, 0])
        sentences.append(df.iloc[i, 1])
    labels = [-1 for _ in range(len(sentences))]
    return sentences, labels

def load_process_data(args, data_dir, dataset, name="train"):
    load_fn = {"util": load_util_sentences}[dataset]
    sentences, labels = load_fn(data_dir, name=name)
    sentences = ["[CLS] " + s for s in sentences]
    tokenizer = get_tokenizer(args.model)
    ids, amasks = get_ids_mask(sentences, tokenizer, args.max_length)
    within_bounds = [ids[i, -1] == 0 for i in range(len(ids))]
    if np.mean(within_bounds) < 1:
        print("{} fraction of examples within context window ({} tokens): {:.3f}".format(name, args.max_length, np.mean(within_bounds)))
    inputs, labels, masks = torch.tensor(ids), torch.tensor(labels), torch.tensor(amasks)

    if "util" in dataset:
        even_mask = [i for i in range(inputs.shape[0]) if i % 2 == 0]
        odd_mask = [i for i in range(inputs.shape[0]) if i % 2 == 1]
        even_inputs, odd_inputs = inputs[even_mask], inputs[odd_mask]
        even_labels, odd_labels = labels[even_mask], labels[odd_mask]
        even_masks, odd_masks = masks[even_mask], masks[odd_mask]
        inputs = torch.stack([even_inputs, odd_inputs], axis=1)
        labels = torch.stack([even_labels, odd_labels], axis=1)
        masks = torch.stack([even_masks, odd_masks], axis=1)

    data = TensorDataset(inputs, masks, labels)
    return data

In [ ]:
# Tensor reshaping functions

def flatten(tensor):
    tensor = torch.cat([tensor[:, 0], tensor[:, 1]])
    return tensor

def unflatten(tensor):
    tensor = torch.stack([tensor[:tensor.shape[0] // 2], tensor[tensor.shape[0] // 2:]], axis=1)
    return tensor

# Convert hyperparams to object for compatibility

class Args:
    def __init__(self, **entries):
        self.__dict__.update(entries)

In [ ]:
# Redefine for each experimental setup before calling main
task, data_dir, train_name, test_name, test_hard_name = None, None, None, None, None

# Training and evaluation loop
def main(hyperparams, verbose=True, save=False):

    args = Args(**hyperparams)
    
    # Load model
    model, optimizer = load_model(args)

    # Load data
    train_and_dev_data = load_process_data(args, data_dir, "util", train_name)
    train_length = int(len(train_and_dev_data)*0.8)
    dev_length = len(train_and_dev_data) - train_length

    train_data, dev_data = random_split(train_and_dev_data, [train_length, dev_length])
    test_data = load_process_data(args, data_dir, "util", test_name)
    if test_hard_name:
      test_hard_data = load_process_data(args, data_dir, "util", test_hard_name)

    train_dataloader = DataLoader(train_data, batch_size=hyperparams['batch_size'] // 2, shuffle=True)
    dev_dataloader = DataLoader(dev_data, batch_size=hyperparams['batch_size'] // 2, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=hyperparams['batch_size'] // 2, shuffle=False)
    if test_hard_name:
      test_hard_dataloader = DataLoader(test_hard_data, batch_size=hyperparams['batch_size'] // 2, shuffle=False)

    # Store dev accuracy per epoch
    best_dev_acc = 0
    best_epoch = None
    best_model = None
    dev_accs = []

    # Train and evaluate the model
    for epoch in range(1, hyperparams['nepochs'] + 1):
        train(model, optimizer, train_dataloader, epoch, verbose=verbose)
        dev_acc = evaluate(model, dev_dataloader)
        dev_accs.append(dev_acc)

        print(f"Epoch {epoch}.\tDev accuracy {dev_acc:.4f}.")

        if dev_acc > best_dev_acc:
          best_dev_acc = dev_acc
          best_epoch = epoch
          best_model = model

    # Store results
    results = hyperparams.copy()
    
    results['best_dev_accuracy'] = best_dev_acc
    results['best_epoch'] = best_epoch

    # Evaluate the best model on the test data
    test_acc = evaluate(best_model, test_dataloader)
    results['test_acc'] = test_acc
    if test_hard_name:
      test_hard_acc = evaluate(best_model, test_hard_dataloader)
      results['test_hard_accuracy'] = test_hard_acc    

    # Save the best model (optional)
    if save:
      save_path = "{}/{}_{}_{}_{}_{}.pkl".format(MODEL_DIR, task, results['model'], results['learning_rate'], results['batch_size'], results['nepochs'])
      print("Saving model to", save_path)
      torch.save(best_model.module.state_dict(), save_path)

    # Save the dev accuracy plot
    if SAVE_FIGS:
      plt.plot(range(1, results['nepochs']+1), dev_accs, color='k', linestyle='-')
      plt.xticks(range(1, results['nepochs']+1))
      plt.legend(['Validation'], loc='lower right')
      plt.ylabel('Accuracy', color='k')
      plt.xlabel('Epoch', color='k')
      plt.title(f'''Accuracy plot (Best val acc: {best_dev_acc:.4f}) \n 
      Model: {hyperparams['model']}, Learning rate: {hyperparams['learning_rate']},  Batch size: {hyperparams['batch_size']}''', color='k')

      plt.savefig(f"{FIG_DIR}/{task}_{hyperparams['model']}_model_{hyperparams['nepochs']}_epochs_{best_dev_acc:.4f}_devacc_ACCPLOT.png", dpi=300, bbox_inches = "tight")
      plt.show()

    return results

def train(model, optimizer, train_dataloader, epoch, log_interval = 10, verbose=False):
    # Set model to training mode
    model.train()
    criterion = torch.nn.BCEWithLogitsLoss()
    ntrain_steps = len(train_dataloader)

    # Loop over each batch from the training set
    for step, batch in enumerate(train_dataloader):

        # Copy data to GPU if needed
        batch = tuple(t.cuda() for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # reshape
        b_input_ids = flatten(b_input_ids)
        b_input_mask = flatten(b_input_mask)

        # Zero gradient buffers
        optimizer.zero_grad()

        # Forward pass
        output = model(b_input_ids, attention_mask=b_input_mask)[0]  # dim 1
        output = unflatten(output)
        diffs = output[:, 0] - output[:, 1]
        loss = criterion(diffs.squeeze(dim=1), torch.ones(diffs.shape[0]).cuda())

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        if step % log_interval == 0 and step > 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, step, ntrain_steps, 100. * step / ntrain_steps, loss))

def evaluate(model, dataloader):
    model.eval()
    cors = []

    for step, batch in enumerate(dataloader):
        # Copy data to GPU if needed
        batch = tuple(t.cuda() for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # reshape
        b_input_ids = flatten(b_input_ids)
        b_input_mask = flatten(b_input_mask)

        # Forward pass
        with torch.no_grad():
            output = model(b_input_ids, attention_mask=b_input_mask)[0]  # dim 1
            output = unflatten(output)
            diffs = output[:, 0] - output[:, 1]
            diffs = diffs.squeeze(dim=1).detach().cpu().numpy()
        cors.append(diffs > 0)

    cors = np.concatenate(cors)
    acc = np.mean(cors)

    return acc

## 1: Train on reformulated dataset

We train on the reformulated dataset to investigate its effect on performance.

### Setup

In [ ]:
task = "util_reformulated_datasets"

data_dir = '4_reformulated_datasets'

grid_search_results_file = f"{RUN_DIR}/{task}_grid_search_results.csv"

train_name = "util_train_no_test_overlap"
test_name = "util_test_easy_matched"
test_hard_name = "util_test_hard_matched"

### Run experiment

In [ ]:
hyperparam_grid = {'model': ["roberta-large"], # ["bert-base-uncased", "bert-large-uncased", "roberta-large", "albert-xxlarge-v2"]
                   'ngpus': [1],
                   'weight_decay': [0.01],
                   'learning_rate': [1e-5, 3e-5],
                   'nepochs': [2, 4],
                   'batch_size': [8, 16],
                   'max_length': [64]}

# Store results of grid search
results_df = pd.DataFrame()

for run_index, hyperparams in enumerate(ParameterGrid(hyperparam_grid)):

  results = main(hyperparams, verbose=True, save=False)

  results_df = results_df.append(results, ignore_index=True)

# Save results of grid search to CSV
results_df.to_csv(grid_search_results_file, mode='a', index=False, header=not os.path.exists(grid_search_results_file))

## 2: Train on unmatching scenario pairs

We train on unmatching scenario pairs to investigate whether it is possible to improve the model's performance on these, since the original dataset contained only matching scenario pairs.



### Setup

In [ ]:
task = "util_unmatching_subsplit"

data_dir = '4_reformulated_datasets/unmatching_subsplit'

grid_search_results_file = f"{RUN_DIR}/{task}_grid_search_results.csv"

train_name = "trainsplit_unmatched_no_test_overlap"
test_name = "testsplit_unmatched"
test_hard_name = None

### Run experiment

In [ ]:
hyperparam_grid = {'model': ["roberta-large"], # ["bert-base-uncased", "bert-large-uncased", "roberta-large", "albert-xxlarge-v2"]
                   'ngpus': [1],
                   'weight_decay': [0.01],
                   'learning_rate': [1e-5, 3e-5],
                   'nepochs': [2, 4],
                   'batch_size': [8, 16],
                   'max_length': [64]}

# Store results of grid search
results_df = pd.DataFrame()

for run_index, hyperparams in enumerate(ParameterGrid(hyperparam_grid)):

  results = main(hyperparams, verbose=True, save=False)

  results_df = results_df.append(results, ignore_index=True)

# Save results of grid search to CSV
results_df.to_csv(grid_search_results_file, mode='a')

## 3: Train on matching and unmatching scenario pairs

We train on a combined dataset of matching and unmatching scenario pairs, to investigate whether the performance on each can be improved.

### Combine datasets

In [ ]:
# Combine training datasets

df1 = pd.read_csv("4_reformulated_datasets/util_train_no_test_overlap.csv", header=None)
df2 = pd.read_csv("4_reformulated_datasets/unmatching_subsplit/trainsplit_unmatched_no_test_overlap.csv", header=None)

df_combined = pd.concat([df1, df2], axis=0)

df_combined.to_csv(f"{OUTPUT_DIR}/util_train_combined_matched_unmatched.csv", index=False, index_label=False, header=False)

# Combine test datasets

df1 = pd.read_csv("4_reformulated_datasets/util_test_easy_matched.csv", header=None)
df2 = pd.read_csv("4_reformulated_datasets/unmatching_subsplit/testsplit_unmatched.csv", header=None)

df_combined = pd.concat([df1, df2], ignore_index=True)

df_combined.to_csv(f"{OUTPUT_DIR}/util_test_combined_matched_unmatched.csv", index=False, index_label=False, header=False)

### Setup

In [ ]:
# Setup

task = "util_combined_split"

data_dir = OUTPUT_DIR

grid_search_results_file = f"{RUN_DIR}/{task}_grid_search_results.csv"

train_name = "util_train_combined_matched_unmatched"
test_name = "util_test_combined_matched_unmatched"
test_hard_name = None

### Run experiment

In [ ]:
hyperparam_grid = {'model': ["roberta-large"], # ["bert-base-uncased", "bert-large-uncased", "roberta-large", "albert-xxlarge-v2"]
                   'ngpus': [1],
                   'weight_decay': [0.01],
                   'learning_rate': [1e-5, 3e-5],
                   'nepochs': [2, 4],
                   'batch_size': [8, 16],
                   'max_length': [64]}

# Store results of grid search
results_df = pd.DataFrame()

for run_index, hyperparams in enumerate(ParameterGrid(hyperparam_grid)):

  results = main(hyperparams, verbose=True, save=False)

  results_df = results_df.append(results, ignore_index=True)

# Save results of grid search to CSV
results_df.to_csv(grid_search_results_file, mode='a', index=False, header=not os.path.exists(grid_search_results_file))

## 4: Assessing calibration of reformulated dataset

We assess the calibration (model uncertainty against accuracy) of the reformulated dataset.

### Generate utility values and labels for all test scenarios

In [ ]:
# Variables to specify

TASK = "util_reformulated_datasets"

MODEL_PATH = f"{MODEL_DIR}/util_reformulated_datasets_roberta-large_1e-05_16_4.pkl"

# "util_test_easy_matched" for easy test dataset
# "util_test_hard_matched" for hard test dataset
# "test_combined_unmatched" for combined unmatched dataset
DATASET = "test_combined_unmatched"

In [ ]:
class Args:
  def __init__(self, task, model, path, ngpus=1, batch_size=16, max_length=64):
    self.task = task
    self.model = model
    self.path = path
    self.ngpus = ngpus
    self.batch_size = batch_size
    self.max_length = max_length

args = Args(task=TASK, model='roberta-large', path=MODEL_PATH)

In [ ]:
# Load all sentences and labels

sentences, labels = load_util_sentences('4_reformulated_datasets', DATASET)

# Load as batchable data

processed_data = load_process_data(args, '4_reformulated_datasets', "util", DATASET)
dataloader = DataLoader(processed_data, batch_size=args.batch_size // 2, shuffle=False)

In [ ]:
def load_model(args, load_path=None, cache_dir=None):
    if cache_dir is not None:
        config = AutoConfig.from_pretrained(args.model, num_labels=1, cache_dir=cache_dir)
    else:
        config = AutoConfig.from_pretrained(args.model, num_labels=1)
    model = AutoModelForSequenceClassification.from_pretrained(args.model, config=config)
    if load_path is not None:
        model.load_state_dict(torch.load(load_path))

    model.cuda()
    model = torch.nn.DataParallel(model, device_ids=[i for i in range(args.ngpus)])

    print('\nPretrained model "{}" loaded'.format(args.model))

    return model

def get_model_results(args, model, dataloader, to_print=False):
    """
    Run the model on the specified dataset, and return pandas dataframe of results
    """
    column_names = ['sentence_good', 'sentence_bad', 'util1', 'util2', 'net_util', 'correctness']
    results_df = pd.DataFrame(columns = column_names)
   
    model.eval()

    for step, batch in enumerate(dataloader):
        # Copy data to GPU if needed
        batch = tuple(t.cuda() for t in batch)

        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # reshape
        b_input_ids = flatten(b_input_ids)
        b_input_mask = flatten(b_input_mask)

        # Forward pass
        with torch.no_grad():
            output = model(b_input_ids, attention_mask=b_input_mask)[0]  # dim 1
            output = unflatten(output).detach().cpu().numpy()
            diffs = output[:, 0] - output[:, 1]
            diffs = diffs.squeeze(axis=1)

            util1 = output[:, 0].flatten()
            util2 = output[:, 1].flatten()
            net_util = (output[:, 0] - output[:, 1]).flatten()
            correctness = (output[:, 0] > output[:, 1]).flatten().astype(int)

            for i in range(len(util1)):

              sentence_idx = 2*step*(args.batch_size // 2)+2*i

              temp_dict = {'sentence_good': sentences[sentence_idx], 'sentence_bad': sentences[sentence_idx+1],
                           'util1': util1[i], 'util2': util2[i], 'net_util': net_util[i], 'correctness': correctness[i]
                           }

              results_df = results_df.append(temp_dict, ignore_index=True)

    return results_df

In [ ]:
# Load the model

model = load_model(args, load_path=MODEL_PATH)

In [ ]:
# Run the model on the specified dataset and save resulting dataframe to Excel

df_preds = get_model_results(args, model, dataloader)

In [ ]:
# Save only if file doesn't already exist (to avoid overwriting)

my_file = Path(f"{RUN_DIR}/{args.task}_{args.model}_{DATASET}_preds.xlsx")
if my_file.is_file() == False:
    df_preds.to_excel(f"{RUN_DIR}/{args.task}_{args.model}_{DATASET}_preds.xlsx")

In [ ]:
# Display the dataframe

df_preds.round(4)

### Adding new columns of interest

In [ ]:
df_easy = pd.read_excel(f"{RUN_DIR}/util_reformulated_datasets_roberta-large_util_test_easy_matched_preds.xlsx", index_col=0)
df_hard = pd.read_excel(f"{RUN_DIR}/util_reformulated_datasets_roberta-large_util_test_hard_matched_preds.xlsx", index_col=0)
df_unmatched = pd.read_excel(f"{RUN_DIR}/util_reformulated_datasets_roberta-large_test_combined_unmatched_preds.xlsx", index_col=0)

In [ ]:
def prepare_stats(df):

    # Utility differences after sigmoid
    df["sigmoid_net_util"] = 1/(1+np.exp(-df["net_util"]))

    # Model's certainty
    df["model_certainty_sigmoid"] = 0.5+(np.abs(df["sigmoid_net_util"]-0.5))

    return df

df_easy = prepare_stats(df_easy)
df_hard = prepare_stats(df_hard)
df_unmatched = prepare_stats(df_unmatched)

In [ ]:
df_hard[['sentence_good', 'sentence_bad', 'util1', 'util2', 'net_util', 'correctness', 'model_certainty_sigmoid']][0:10].round(4)

### Assessing calibration

In [ ]:
def split_in_bins(predictions, confidence):
    num_bins = 5
    l = np.linspace(0.5,1,num_bins+1)
    bins = np.linspace(0.5,.9,num_bins)+.05

    conf = []
    acc = []
    num_in_bins = []

    for ind, (lower,upper) in enumerate(zip(l[:-1], l[1:])):
        indxs = np.where((confidence<=upper) & (confidence>lower)) # B_m

        this_bin_pred = predictions[indxs]
        this_bin_conf = confidence[indxs]

        # Get average confidence
        avg_conf = np.mean(this_bin_conf)

        # Get average accuracy
        avg_acc = np.mean(this_bin_pred)
        conf.append(avg_conf)
        acc.append(avg_acc)
        num_in_bins.append(len(this_bin_pred))
    
    return conf, acc, bins, num_in_bins

def get_ECE(confidence, accuracy, num_in_bins):
  '''
  condifence: list of conf(B_m)
  accuracy: list of acc(B_m)

  num_in_bins: number of samples in each bin
  '''
  assert len(confidence) == len(accuracy)

  num_in_bins = np.asarray(num_in_bins)
  n = num_in_bins.sum() # Total number of samples
  ECE = 0
  for i in range(len(confidence)):
    ECE += (num_in_bins[i]/(n)) * np.abs(accuracy[i] - confidence[i])

  return ECE

def plot_reliability_diagram(accuracy_easy, bins_easy, accuracy_hard, bins_hard, accuracy_unmatched, bins_unmatched):
    accuracy = (accuracy_easy, accuracy_hard, accuracy_unmatched)
    bins = (bins_easy, bins_hard, bins_unmatched)
    width=0.1
    fig, ax = plt.subplots(figsize=(13,5), nrows=1, ncols=3)
    fig.suptitle("Assessing calibration of model certainty against accuracy\n(non-Bayesian RoBERTa-large)", fontsize=14, fontweight='bold')
    for i in range(3):
        ax[i].bar(bins[i], accuracy[i], width=width, color='k', edgecolor='black')
        ax[i].plot(np.linspace(0.5,1,6),np.linspace(0.5,1,6),linestyle='--', color='red')
        ax[i].set_ylabel("Accuracy")
        ax[i].set_xlabel("Model certainty")
        if i==0:
            ax[i].set_title("Easy test dataset")
        if i==1:
            ax[i].set_title("Hard test dataset")
        if i==2:
            ax[i].set_title("Unmatched test dataset")
    fig.tight_layout(rect=[0, 0, 1, 0.90])
    fig.legend(['Perfect certainty','Model certainty'],loc=(0.75,0.15), facecolor="white")
    if SAVE_FIGS==True:
        fig.savefig(f"{FIG_DIR}/hist_accuracy_v_modelcertainty", dpi=250)
    fig.show()

In [ ]:
conf, acc, bins, num_in_bins = split_in_bins(df_easy['correctness'].to_numpy(), df_easy['model_certainty_sigmoid'].to_numpy())
conf_hard, acc_hard, bins_hard, num_in_bins_hard = split_in_bins(df_hard['correctness'].to_numpy(), df_hard['model_certainty_sigmoid'].to_numpy())
conf_unmatched, acc_unmatched, bins_unmatched, num_in_bins_unmatched = split_in_bins(df_unmatched['correctness'].to_numpy(), df_unmatched['model_certainty_sigmoid'].to_numpy())

ece_easy = get_ECE(conf, acc, num_in_bins)
print(f"ECE easy test dataset: {ece_easy:.4f}")

ece_hard = get_ECE(conf_hard, acc_hard, num_in_bins_hard)
print(f"ECE hard test dataset: {ece_hard:.4f}")

ece_unmatched = get_ECE(conf_unmatched, acc_unmatched, num_in_bins_unmatched)
print(f"ECE unmatched test dataset: {ece_unmatched:.4f}")

plot_reliability_diagram(acc, bins, acc_hard, bins_hard, acc_unmatched, bins_unmatched)